# Export sparse feature embedding

现在我们了解TensorNet保存的`sparse_table`目录中的数据内容及格式，以方便导出成字典供在线使用。

In [2]:
MODEL_DIR = '/tmp/wide-deep-test/model'

In [3]:
! tree $MODEL_DIR/sparse_table

/tmp/wide-deep-test/model/sparse_table
├── 0
│   └── rank_0
│       ├── sparse_block_0.gz
│       ├── sparse_block_1.gz
│       ├── sparse_block_2.gz
│       ├── sparse_block_3.gz
│       ├── sparse_block_4.gz
│       ├── sparse_block_5.gz
│       ├── sparse_block_6.gz
│       └── sparse_block_7.gz
└── 1
    └── rank_0
        ├── sparse_block_0.gz
        ├── sparse_block_1.gz
        ├── sparse_block_2.gz
        ├── sparse_block_3.gz
        ├── sparse_block_4.gz
        ├── sparse_block_5.gz
        ├── sparse_block_6.gz
        └── sparse_block_7.gz

4 directories, 16 files


`sparse_table`目录下会有多个目录，目录的个数由`tn.layers.EmbeddingFeatures`调用的次数确定，在我们tutorial中的例子中`wide`部分调用了一次，`deep`中调用了一次，那么便有两个目录。目录的名称依照调用的先后顺序从0开始命名，这个例子里面名称为`0`的目录下保存的是`wide`部分的embedding数据，为`1`的目录下保存的是`deep`部分的embedding数据。子目录中分别保存了每个节点的数据，以`rank_*`命名，在我们这个例子中使用单节点运行，所以只有`rank_0`。在每个节点的数据中分别会有8个block，每个block中的数据格式一致。

## sparse embedding数据格式

sparse block中的数据全部按`tab`键分隔，可以通过下面命令查看

In [4]:
!cat $MODEL_DIR/sparse_table/1/rank_0/sparse_block_0.gz | gunzip | head -10

opt_name:AdaGrad
dim:8
63927	8	0.0262204	-0.0414651	0.0461724	0.0260017	0.0613893	-0.0325357	0.0551388	-0.00449165	0.1	1	0.98
61514	8	0.0209959	-0.0770077	-0.0248773	0.016569	0.0071595	0.0478604	0.0274112	0.0725264	0.1	1	0.98
56580	8	0.00379409	-0.0978684	0.0398026	-0.0278145	-0.00481733	-0.00540131	-0.0336508	0.0101625	0.1	1	0.98
51391	8	0.0342308	-0.00472191	-0.0216889	0.0170641	0.00393812	-0.007634	0.0107123	0.0233057	0.1	1	0.98
41190	8	-0.0501618	-0.0142409	-0.0427884	-0.064903	0.0422692	-0.0217611	0.0552286	0.0355111	0.1	1	0.98
35619	8	0.0202833	-0.00314469	-0.00274868	-0.0165426	0.00438455	-0.0344267	0.0173564	0.0341289	0.1	1	0.98
31504	8	0.0344835	-0.00100818	0.0224287	-0.0199555	-0.0218565	-0.0594322	-0.0253813	0.0232026	0.1	1	0.98
25596	8	-0.0139298	-0.0488882	0.0384313	0.0378851	0.00378205	0.0485842	-0.080289	-0.0162278	0.1	1	0.98

gzip: stdout: Broken pipe
cat: write error: Broken pipe


文件的头两行中写了优化器的名称及用户设置的embedding的维度，之后的每一行都是具体的数据了。每一行的格式如下：

| sign    | dimension | embedding1 | embedding2 | ... | embeddingN | 优化器超参1| ...      | 优化器超参N | version | show_count |
|---------|-----------|------------|------------|-----|------------|------------|----------|-------------| --------| -----------|
| 63927  | 8         | 0.0262204  | -0.0414651 | ... |-0.00449165 | 0.1        | ...      |  0          | 1       | 0.98       | 


每个优化器上所带的超参可能会不同，具体的可以参考`core/ps/optimizer/ada_grad_kernel.cc`和`core/ps/optimizer/adam_kernel.cc`中的实现，我们在线预估真正关注的只是前面的embedding数据

## 制作在线字典

上面的sparse table的格式比较明确，那么使用mapreduce或者spark很容易截取出我们需要的数据，以`sign`为key，embedding数据为value保存成字典就可以了。

在实际中，sparse embedding的数据会非常大，很可能会接近TB，我们内部是使用一个分布式哈希表将这个数据存储在多台机器上，然后在线使用rpc获取具体的embedding数据。如果没有精力实现分布式哈希表，则可以参考数据的最后一列`show_count`，将出现次数较少的特征砍掉以缩减字典的大小。